In [1]:
import numpy as np 

from itertools import product

from strawberryfields.decompositions import takagi 
from thewalrus.symplectic import squeezing, passive_transformation

from thewalrus.quantum import Amat, state_vector

### Turning PYTHEUS HALO graphs into a Terry friendly format
and expressing their state in the Fock basis

In [2]:
# first write down the graphs weighted adjacency matrix (I did this by hand)

B = 0.3 * np.array([    # weight of 0.3 was found through coarse hand optimisation of success probability
    [0,0,0,-1,1,0,0,0],
    [0,0,1,0,0,0,0,0,],
    [0,1,0,1,1,0,0,0],
    [-1,0,1,0,0,1,0,1],
    [1,0,1,0,0,1,0,-1],
    [0,0,0,1,1,0,1,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,1,-1,0,0,0]
])

We could jump straight to the Fock basis expression from the adjacency matrix by using results from Gaussian quantum optics e.g. eq. 66 of https://arxiv.org/abs/2209.06069 or eq. 105 of https://arxiv.org/abs/1811.09597. 

However, for convienience, we will define the states covariance matrix first which let's use leverage some open source tools.


In [3]:
# decompose the matrix into squeezing parameters and a linear optical unitary
tanhr, U = takagi(B) 
sq = -np.arctanh(tanhr)

# calculate covariance matrix after squeezing
S = squeezing(sq)
cov = S @ S.T 

# calculate covariance matrix after linear optical unitary
mu = np.zeros(16)
mu, cov = passive_transformation(mu, cov, U)

# express state in the Fock basis, after heralding on 1 photon in modes 2,3,4,5 (counting from 0)
cutoff=5
ket = state_vector(mu, cov, post_select={2:1,3:1,4:1,5:1}, cutoff=cutoff)

/Users/jbulmer/mambaforge/envs/mess/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
/Users/jbulmer/mambaforge/envs/mess/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/jbulmer/mambaforge/envs/mess/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
/Users/jbulmer/mambaforge/envs/mess/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det


In [4]:
p_succ = 0
for term in product(range(cutoff), repeat=4):
    amp = ket[term]
    if abs(amp) > 1e-14:
        print(amp, '*', f"|{str(term)[1:-1]}>")
        p_succ += abs(amp)**2
print()
print(f'success probability = {p_succ}')

(0.1018742754575462+0j) * |0, 0, 0, 0>
(-0.006483239190404748+0j) * |0, 1, 1, 2>
(0.009168684791179169+0j) * |1, 1, 1, 1>
(-0.006483239190404751+0j) * |2, 1, 1, 0>

success probability = 0.0105464975616


Terry is too ~~stuborn~~ ~~old~~ cool to use python, so we export the squeezing and unitary as a matlab file

In [5]:
from scipy.io import savemat, loadmat

savemat('squeezing_and_unitary.mat', 
    {"r" : sq, "U" : U})